In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [24]:
#Problem 1

# Parameters
batch_size = 128
layer_size = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    #beta = tf.Variable(0.01)

    # Logits = concatenation of operations: ((D*W1+b1)*ReLU*W2)+b2
    logits = tf.matmul(tf_train_dataset, W1) + b1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    beta * 0.5 * tf.nn.l2_loss(W1))
   
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # Prediction = softmax(Logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, W1) + b1)
    test_prediction =  tf.nn.softmax(tf.matmul(tf_test_dataset, W1 ) + b1)

In [25]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 21.365869522094727
Minibatch accuracy: 3.1
Validation accuracy: 7.7
Minibatch loss at step 500: 2.445897102355957
Minibatch accuracy: 73.4
Validation accuracy: 76.7
Minibatch loss at step 1000: 2.1249451637268066
Minibatch accuracy: 73.4
Validation accuracy: 77.8
Minibatch loss at step 1500: 1.4479525089263916
Minibatch accuracy: 81.2
Validation accuracy: 78.5
Minibatch loss at step 2000: 1.1259050369262695
Minibatch accuracy: 78.9
Validation accuracy: 80.0
Minibatch loss at step 2500: 0.828095555305481
Minibatch accuracy: 85.9
Validation accuracy: 80.9
Minibatch loss at step 3000: 0.9189773797988892
Minibatch accuracy: 82.0
Validation accuracy: 80.9
Test accuracy: 87.8


In [29]:
# Problem 2

# Parameters
batch_size = 128
layer_size = 1024
beta = 0.001

reduced_batch_number = 3
reduced_datset = train_dataset[0:reduced_batch_number * batch_size]
reduced_labels = train_labels[0:reduced_batch_number * batch_size]

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    #beta = tf.Variable(0.01)

    # Logits = concatenation of operations: ((D*W1+b1)*ReLU*W2)+b2
    logits = tf.matmul(tf_train_dataset, W1) + b1
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    beta * 0.5 * tf.nn.l2_loss(W1))
   
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # Prediction = softmax(Logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, W1) + b1)
    test_prediction =  tf.nn.softmax(tf.matmul(tf_test_dataset, W1 ) + b1)

In [30]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (reduced_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = reduced_datset[offset:(offset + batch_size), :]
        batch_labels = reduced_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 17.188066482543945
Minibatch accuracy: 12.5
Validation accuracy: 13.6
Minibatch loss at step 500: 1.1433937549591064
Minibatch accuracy: 100.0
Validation accuracy: 61.6
Minibatch loss at step 1000: 0.8932042121887207
Minibatch accuracy: 100.0
Validation accuracy: 63.0
Minibatch loss at step 1500: 0.699794590473175
Minibatch accuracy: 100.0
Validation accuracy: 64.7
Minibatch loss at step 2000: 0.5495110750198364
Minibatch accuracy: 100.0
Validation accuracy: 65.8
Minibatch loss at step 2500: 0.4327054023742676
Minibatch accuracy: 100.0
Validation accuracy: 66.9
Minibatch loss at step 3000: 0.34194430708885193
Minibatch accuracy: 100.0
Validation accuracy: 67.8
Test accuracy: 74.0


In [37]:
# Problem mix between 1 and 2

# Parameters
batch_size = 128
layer_size = 1024
beta = 0.001

reduced_batch_number = 3
#num_data = reduced_batch_number * batch_size
num_data = train_dataset.shape[0]
reduced_datset = train_dataset[0:num_data]
reduced_labels = train_labels[0:num_data]

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer_size]))
    W2 = tf.Variable(tf.truncated_normal([layer_size, num_labels]))
    b1 = tf.Variable(tf.zeros([layer_size]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Logits = concatenation of operations: ((D*W1+b1)*ReLU*W2)+b2
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1), W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
   
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # Prediction = softmax(Logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1 ) + b1), W2) + b2)


In [38]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (reduced_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = reduced_datset[offset:(offset + batch_size), :]
        batch_labels = reduced_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 790.9886474609375
Minibatch accuracy: 7.0
Validation accuracy: 33.4
Minibatch loss at step 500: 195.89181518554688
Minibatch accuracy: 75.0
Validation accuracy: 80.9
Minibatch loss at step 1000: 115.8136978149414
Minibatch accuracy: 81.2
Validation accuracy: 79.8
Minibatch loss at step 1500: 68.55885314941406
Minibatch accuracy: 85.2
Validation accuracy: 81.7
Minibatch loss at step 2000: 41.45638656616211
Minibatch accuracy: 85.2
Validation accuracy: 85.1
Minibatch loss at step 2500: 25.039241790771484
Minibatch accuracy: 90.6
Validation accuracy: 86.1
Minibatch loss at step 3000: 15.45228099822998
Minibatch accuracy: 87.5
Validation accuracy: 86.7
Test accuracy: 92.8


In [42]:
# Problem 3

# Parameters
batch_size = 128
layer_size = 1024
beta = 0.001
dropout_data = 0.5

reduced_batch_number = 3
#num_data = reduced_batch_number * batch_size
num_data = train_dataset.shape[0]
reduced_datset = train_dataset[0:num_data]
reduced_labels = train_labels[0:num_data]

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer_size]))
    W2 = tf.Variable(tf.truncated_normal([layer_size, num_labels]))
    b1 = tf.Variable(tf.zeros([layer_size]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Logits = concatenation of operations: ((D*W1+b1)*ReLU*W2)+b2
    logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1), dropout_data), W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
   
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # Prediction = softmax(Logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1 ) + b1), W2) + b2)

In [43]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (reduced_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = reduced_datset[offset:(offset + batch_size), :]
        batch_labels = reduced_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 876.982421875
Minibatch accuracy: 7.0
Validation accuracy: 28.8
Minibatch loss at step 500: 207.09588623046875
Minibatch accuracy: 70.3
Validation accuracy: 81.0
Minibatch loss at step 1000: 118.63298034667969
Minibatch accuracy: 71.9
Validation accuracy: 79.8
Minibatch loss at step 1500: 71.0679931640625
Minibatch accuracy: 74.2
Validation accuracy: 82.2
Minibatch loss at step 2000: 42.00693893432617
Minibatch accuracy: 81.2
Validation accuracy: 83.8
Minibatch loss at step 2500: 25.04107093811035
Minibatch accuracy: 86.7
Validation accuracy: 84.5
Minibatch loss at step 3000: 15.457677841186523
Minibatch accuracy: 82.8
Validation accuracy: 85.4
Test accuracy: 91.6


In [61]:
# Problem 3

# Parameters
batch_size = 256
layer_size = 1024
beta = 0.001
dropout_data = 0.5
decay_steps = 100000
decay_rate = 0.98

reduced_batch_number = 3
#num_data = reduced_batch_number * batch_size
num_data = train_dataset.shape[0]
reduced_datset = train_dataset[0:num_data]
reduced_labels = train_labels[0:num_data]

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer_size]))
    #W2 = tf.Variable(tf.truncated_normal([layer_size, layer_size]))
    W2 = tf.Variable(tf.truncated_normal([layer_size, num_labels]))
    b1 = tf.Variable(tf.zeros([layer_size]))
    #b2 = tf.Variable(tf.zeros([layer_size]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Logits = concatenation of operations: ((D*W1+b1)*ReLU*W2)+b2
    h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1), dropout_data)
    #h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h1, W2) + b2), dropout_data)
    logits = tf.matmul(h1, W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss += beta * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
    #  + tf.nn.l2_loss(W3)
   
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # Prediction = softmax(Logits)
    h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
    #h2_valid = tf.nn.relu(tf.matmul(h1_valid, W2) + b2)
    valid_prediction = tf.nn.softmax(tf.matmul(h1_valid, W2) + b2)
    h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
    #h2_test = tf.nn.relu(tf.matmul(h1_test, W2) + b2)
    test_prediction =  tf.nn.softmax(tf.matmul(h1_test, W2) + b2)

In [62]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (reduced_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = reduced_datset[offset:(offset + batch_size), :]
        batch_labels = reduced_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 866.0966186523438
Minibatch accuracy: 7.4
Validation accuracy: 36.5
Minibatch loss at step 500: 197.94317626953125
Minibatch accuracy: 77.7
Validation accuracy: 82.0
Minibatch loss at step 1000: 113.82234191894531
Minibatch accuracy: 77.3
Validation accuracy: 82.2
Minibatch loss at step 1500: 68.5621109008789
Minibatch accuracy: 77.3
Validation accuracy: 83.3
Minibatch loss at step 2000: 41.3434944152832
Minibatch accuracy: 80.1
Validation accuracy: 84.7
Minibatch loss at step 2500: 25.24720573425293
Minibatch accuracy: 81.2
Validation accuracy: 86.0
Minibatch loss at step 3000: 15.50088119506836
Minibatch accuracy: 81.2
Validation accuracy: 86.6
Test accuracy: 92.7
